## Imports

In [50]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# **Data Preparation**

## **1. Carregar e Processar Dados de Produção**

In [65]:
df_producao_raw = pd.read_csv("../data/raw/ReparticaoProducao.csv", na_values=-990, sep=';', skiprows=2)
df_producao_raw.columns = df_producao_raw.columns.str.strip()
df_producao = df_producao_raw[['Data e Hora', 'Eólica']].copy()
df_producao['Data e Hora'] = pd.to_datetime(df_producao['Data e Hora'])
df_producao.set_index('Data e Hora', inplace=True)

# Agregar para diário
df_producao_diaria = df_producao.resample('D').sum().reset_index()
df_producao_diaria.rename(columns={'Data e Hora': 'Data', 'Eólica': 'Eólica_Total_Dia'}, inplace=True) # Renomear para clareza

##  **2. Carregar e Processar Dados Meteorológicos** 


In [68]:
# Função para carregar, tratar NaNs e preparar data
def process_meteo_data(filepath, nan_strategy='median', circular_median_func=None):
    print(f"Processando: {filepath}")
    df = pd.read_csv(filepath, na_values=-990, sep=';', decimal='.')
    df.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'}, inplace=True)
    df['Data'] = pd.to_datetime(df[['year', 'month', 'day']])

    station_cols = [col for col in df.columns if col.isdigit()]

    if nan_strategy == 'median':
        for col in station_cols:
            if df[col].isnull().any():
                median_val = df[col].median()
                df.fillna({col: median_val}, inplace=True)
    elif nan_strategy == 'mean':
         for col in station_cols:
            if df[col].isnull().any():
                mean_val = df[col].mean()
                df.fillna({col: mean_val}, inplace=True)
    elif nan_strategy == 'interpolate_median': # Estratégia mista para Intensidade
         missing_percent = (df.isnull().mean() * 100)
         for col in station_cols:
             if df[col].isnull().any():
                 if missing_percent[col] > 30:
                     df[col] = df[col].interpolate(method='linear', limit_direction='both')
                     median_val = df[col].median() # Recalcular mediana pós-interpolação
                     df.fillna({col: median_val}, inplace=True) # Preencher bordas
                 else:
                     median_val = df[col].median()
                     df.fillna({col: median_val}, inplace=True)
    elif nan_strategy == 'circular_median' and circular_median_func:
         for col in station_cols:
             if df[col].isnull().any():
                # Calcular mediana circular para a coluna específica (ignorando NaNs no cálculo)
                median_circ = circular_median_func(df[col].dropna().values)
                df.fillna({col: median_circ}, inplace=True)
    else:
        print(f"Estratégia NaN desconhecida ou função em falta para: {filepath}")

    print(f"Valores ausentes restantes em {filepath.split('/')[-1]}: {df[station_cols].isnull().sum().sum()}")
    return df[['Data'] + station_cols]

# Função para calcular a mediana circular
def circular_median(values):
    radians = np.deg2rad(values)
    median_radians = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    return np.rad2deg(median_radians) % 360


In [69]:
# Processar cada ficheiro meteorológico
df_intensidade = process_meteo_data("../data/raw/IntensidadeMediaVento10m.csv", nan_strategy='interpolate_median')
df_temperatura = process_meteo_data("../data/raw/TemperaturaMedia.csv", nan_strategy='mean') # Usando média como no teu código original
df_direcao = process_meteo_data("../data/raw/DirecaoMediaVento10m.csv", nan_strategy='circular_median', circular_median_func=circular_median)

Processando: ../data/raw/IntensidadeMediaVento10m.csv
Valores ausentes restantes em IntensidadeMediaVento10m.csv: 0
Processando: ../data/raw/TemperaturaMedia.csv
Valores ausentes restantes em TemperaturaMedia.csv: 0
Processando: ../data/raw/DirecaoMediaVento10m.csv
Valores ausentes restantes em DirecaoMediaVento10m.csv: 0


## **3. Calcular Médias Nacionais Diárias (Agregadas)**

In [70]:
station_cols_intensidade = [col for col in df_intensidade.columns if col.isdigit()]
station_cols_temperatura = [col for col in df_temperatura.columns if col.isdigit()]
station_cols_direcao = [col for col in df_direcao.columns if col.isdigit()]

# Verificar se as colunas de estação são as mesmas (importante para vector avg)
if not (station_cols_intensidade == station_cols_temperatura == station_cols_direcao):
    print("AVISO: Colunas de estação não coincidem entre os ficheiros meteorológicos!")
    # Decidir como lidar com isso - usar interseção, ou parar? Por agora, usamos as da intensidade.
    station_cols = station_cols_intensidade
else:
    station_cols = station_cols_intensidade

# Média simples para Intensidade e Temperatura
df_intensidade_agg = pd.DataFrame({
    'Data': df_intensidade['Data'],
    'Intensidade_Media': df_intensidade[station_cols].mean(axis=1)
})
df_temperatura_agg = pd.DataFrame({
    'Data': df_temperatura['Data'],
    'Temperatura_Media': df_temperatura[station_cols].mean(axis=1)
})

# Média vetorial para Direção
dd_rad = np.deg2rad(df_direcao[station_cols])
intensidade_estacoes = df_intensidade[station_cols] # Usar intensidade por estação como magnitude

# Componentes U (zonal, Este-Oeste) e V (meridional, Norte-Sul)
# Convenção meteorológica: U positivo = vento de Oeste, V positivo = vento de Sul
# ângulo 0 = Norte, 90 = Este, 180 = Sul, 270 = Oeste
# U = -Intensidade * sin(Direção_rad)
# V = -Intensidade * cos(Direção_rad)
u_component = -intensidade_estacoes * np.sin(dd_rad)
v_component = -intensidade_estacoes * np.cos(dd_rad)

u_medio = u_component.mean(axis=1)
v_medio = v_component.mean(axis=1)

# Calcular direção média nacional a partir das componentes médias U e V
# Adicionar 180 graus ao arctan2 e usar módulo 360 para converter de volta para convenção meteorológica
direcao_media_nacional = (np.rad2deg(np.arctan2(u_medio, v_medio)) + 180) % 360

# Calcular componentes sin/cos da direção média nacional (para feature engineering)
df_direcao_agg = pd.DataFrame({
    'Data': df_direcao['Data'],
    'Direcao_Media': direcao_media_nacional
    # As componentes sin/cos serão calculadas mais tarde no DataFrame final
})


## **4. Juntar Todos os Dados Diários**

In [71]:
df_final = df_producao_diaria.merge(df_intensidade_agg, on='Data', how='inner')
df_final = df_final.merge(df_temperatura_agg, on='Data', how='inner')
df_final = df_final.merge(df_direcao_agg, on='Data', how='inner')

# Renomear coluna alvo para consistência com o notebook de modelação
df_final.rename(columns={'Eólica_Total_Dia': 'Eólica'}, inplace=True)

# Ordenar por data (importante!)
df_final = df_final.sort_values('Data').reset_index(drop=True)

print(f"DataFrame combinado inicial: {df_final.shape[0]} linhas.")

DataFrame combinado inicial: 4017 linhas.


## **5. Engenharia de Features Avançada**

In [73]:
# 5.1 Features Baseadas na Data
df_final['mes'] = df_final['Data'].dt.month
df_final['dia_da_semana'] = df_final['Data'].dt.dayofweek
df_final['dia_do_ano'] = df_final['Data'].dt.dayofyear
df_final['semana_do_ano'] = df_final['Data'].dt.isocalendar().week.astype(int)
df_final['trimestre'] = df_final['Data'].dt.quarter
df_final['eh_fim_de_semana'] = df_final['dia_da_semana'].isin([5, 6]).astype(int)

# 5.2 Codificação Cíclica (Vento e Data)
df_final['vento_sin'] = np.sin(np.radians(df_final['Direcao_Media']))
df_final['vento_cos'] = np.cos(np.radians(df_final['Direcao_Media']))
df_final['dia_semana_sin'] = np.sin(2 * np.pi * df_final['dia_da_semana'] / 7)
df_final['dia_semana_cos'] = np.cos(2 * np.pi * df_final['dia_da_semana'] / 7)
df_final['mes_sin'] = np.sin(2 * np.pi * df_final['mes'] / 12)
df_final['mes_cos'] = np.cos(2 * np.pi * df_final['mes'] / 12)
df_final['dia_ano_sin'] = np.sin(2 * np.pi * df_final['dia_do_ano'] / 366)
df_final['dia_ano_cos'] = np.cos(2 * np.pi * df_final['dia_do_ano'] / 366)

# 5.3 Lag Features (Atrasos)
lags_eolica = [1, 2, 3, 7, 14] # Adicionei lag 14
lags_meteo = [1, 2, 3, 7]

for lag in lags_eolica:
    df_final[f'Eólica_lag{lag}'] = df_final['Eólica'].shift(lag)

for lag in lags_meteo:
    df_final[f'Intensidade_Media_lag{lag}'] = df_final['Intensidade_Media'].shift(lag)
    df_final[f'Temperatura_Media_lag{lag}'] = df_final['Temperatura_Media'].shift(lag)
    df_final[f'vento_sin_lag{lag}'] = df_final['vento_sin'].shift(lag)
    df_final[f'vento_cos_lag{lag}'] = df_final['vento_cos'].shift(lag)

# 5.4 Rolling Window Features (Janelas Deslizantes)
window_sizes = [3, 7, 14]
for window in window_sizes:
    # Eólica
    df_final[f'Eolica_roll_mean_{window}'] = df_final['Eólica'].shift(1).rolling(window=window, min_periods=1).mean()
    df_final[f'Eolica_roll_std_{window}'] = df_final['Eólica'].shift(1).rolling(window=window, min_periods=1).std()
    # Intensidade Média
    df_final[f'Intensidade_Media_roll_mean_{window}'] = df_final['Intensidade_Media'].shift(1).rolling(window=window, min_periods=1).mean()
    df_final[f'Intensidade_Media_roll_std_{window}'] = df_final['Intensidade_Media'].shift(1).rolling(window=window, min_periods=1).std()
    # Temperatura Média (opcional)
    df_final[f'Temperatura_Media_roll_mean_{window}'] = df_final['Temperatura_Media'].shift(1).rolling(window=window, min_periods=1).mean()
    df_final[f'Temperatura_Media_roll_std_{window}'] = df_final['Temperatura_Media'].shift(1).rolling(window=window, min_periods=1).std()

## **6. Tratar NaNs Finais e guardar o dataset**

In [ ]:
# Preencher NaNs no início usando backfill
cols_com_lags_roll = [col for col in df_final.columns if '_lag' in col or '_roll_' in col]
df_final[cols_com_lags_roll] = df_final[cols_com_lags_roll].fillna(method='bfill')

# Verificar se ainda existem NaNs
print("\nVerificação final de NaNs:")
nans_finais = df_final.isnull().sum()
print(nans_finais[nans_finais > 0]) # Mostra apenas colunas com NaNs
if nans_finais.sum() == 0:
    print("Nenhum NaN restante.")
else:
    print("ATENÇÃO: Ainda existem NaNs. Verificar o preenchimento.")
    # Poderias precisar de um .fillna(0) ou outra estratégia se o bfill não for suficiente
    # df_final.fillna(0, inplace=True) # Exemplo: preencher restantes com 0

# Opcional: Remover colunas que não serão usadas como features
cols_to_drop_final = ['Direcao_Media', 'mes', 'dia_da_semana', 'dia_do_ano'] # Manter Data e Eólica
# df_final = df_final.drop(columns=cols_to_drop_final)

# Garantir que o diretório existe
os.makedirs('../data/processed', exist_ok=True)

# Salvar o dataset final pronto para ML
output_path = '../data/processed/agg_data_ml.csv'
df_final.to_csv(output_path, index=False)
print(f"\nDataFrame final para ML salvo em: {output_path}")

# --- Mostrar Resultado Final ---
print("\n--- Primeiras 5 linhas do DataFrame final ---")
print(df_final.head())
print("\n--- Últimas 5 linhas do DataFrame final ---")
print(df_final.tail())
print("\n--- Colunas Finais ---")
print(df_final.columns.tolist())
print(f"\nDimensões finais do DataFrame: {df_final.shape}")


Verificação final de NaNs:
Series([], dtype: int64)
Nenhum NaN restante.

DataFrame final para ML salvo em: ../data/processed/agg_data_ml.csv

--- Primeiras 5 linhas do DataFrame final ---
        Data    Eólica  Intensidade_Media  Temperatura_Media  Direcao_Media  \
0 2013-01-01   89358.2           1.988889           9.488889     327.262439   
1 2013-01-02  118865.5           1.872222           8.683333      31.852420   
2 2013-01-03  232257.7           2.972222           9.277778      79.706195   
3 2013-01-04  149543.7           2.377778           8.761111      88.710336   
4 2013-01-05   23830.8           1.605556           8.600000      83.465882   

   mes  dia_da_semana  dia_do_ano  semana_do_ano  trimestre  ...  \
0    1              1           1              1          1  ...   
1    1              2           2              1          1  ...   
2    1              3           3              1          1  ...   
3    1              4           4              1          1  ..

C:\Users\Tiago\AppData\Local\Temp\ipykernel_13116\1584817216.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final[cols_com_lags_roll] = df_final[cols_com_lags_roll].fillna(method='bfill')


# 1. Conversão de Data

# **Tratar Valores Ausentes**

## Reparticao da Producao de Energia

In [51]:
df_producao = pd.read_csv("../data/raw/ReparticaoProducao.csv", na_values=-990, sep=';',skiprows=2)

# Resumo dos valores ausentes
missing_data_producao = pd.DataFrame({
    'Total Ausentes': df_producao.isnull().sum(),
    'Percentual (%)': (df_producao.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data_producao)


Resumo das colunas com valores ausentes:
                               Total Ausentes  Percentual (%)
Data e Hora                                 0             0.0
Hídrica                                     0             0.0
Eólica                                      0             0.0
Solar                                       0             0.0
Biomassa                                    0             0.0
Ondas                                       0             0.0
Gás Natural - Ciclo Combinado               0             0.0
Gás natural - Cogeração                     0             0.0
Carvão                                      0             0.0
Outra Térmica                               0             0.0
Importação                                  0             0.0
Exportação                                  0             0.0
Bombagem                                    0             0.0
Injeção de Baterias                         0             0.0
Consumo Baterias            

## Intensidade Media do Vento a 10m

In [52]:
# Importando o arquivo CSV, tratando -990 como ausente
df_intesidadeVento10 = pd.read_csv("../data/raw/IntensidadeMediaVento10m.csv", na_values=-990, sep=';', decimal='.')

# Resumo dos valores ausentes
missing_data = pd.DataFrame({
    'Total Ausentes': df_intesidadeVento10.isnull().sum(),
    'Percentual (%)': (df_intesidadeVento10.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data)



Resumo das colunas com valores ausentes:
         Total Ausentes  Percentual (%)
ANO                   0            0.00
MES                   0            0.00
DIA                   0            0.00
1200551            1981           49.32
1210622            1525           37.96
1200567              14            0.35
1200575              57            1.42
1200545             283            7.05
1210702             663           16.50
1200560              83            2.07
1210683             190            4.73
1200548              57            1.42
1200570               1            0.02
1210718             191            4.75
1210734              52            1.29
1200571             226            5.63
1200579              32            0.80
1210770             192            4.78
1200558              28            0.70
1200562             471           11.73
1200554             352            8.76


## Direcao Media do Vento a 10m

In [53]:
df_direcaoVento10 = pd.read_csv("../data/raw/DirecaoMediaVento10m.csv", na_values=-990, sep=';', decimal='.')

# Resumo dos valores ausentes
missing_data = pd.DataFrame({
    'Total Ausentes': df_direcaoVento10.isnull().sum(),
    'Percentual (%)': (df_direcaoVento10.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data)


Resumo das colunas com valores ausentes:
         Total Ausentes  Percentual (%)
ANO                   0            0.00
MES                   0            0.00
DIA                   0            0.00
1200551            2540           63.23
1210622            1090           27.13
1200567              36            0.90
1200575              47            1.17
1200545             617           15.36
1210702             665           16.55
1200560              85            2.12
1210683             191            4.75
1200548              63            1.57
1200570               1            0.02
1210718             207            5.15
1210734              62            1.54
1200571            1052           26.19
1200579              36            0.90
1210770             192            4.78
1200558              36            0.90
1200562             287            7.14
1200554            1026           25.54


## Temperatura Media

In [54]:
df_temperaturamedia = pd.read_csv("../data/raw/TemperaturaMedia.csv", na_values=-990, sep=';', decimal='.')

# Resumo dos valores ausentes
missing_data = pd.DataFrame({
    'Total Ausentes': df_temperaturamedia.isnull().sum(),
    'Percentual (%)': (df_temperaturamedia.isnull().mean() * 100).round(2)
})

print("\nResumo das colunas com valores ausentes:")
print(missing_data)


Resumo das colunas com valores ausentes:
         Total Ausentes  Percentual (%)
ANO                   0            0.00
MES                   0            0.00
DIA                   0            0.00
1200551             288            7.17
1210622             240            5.97
1200567              14            0.35
1200575              49            1.22
1200545             512           12.75
1210702              98            2.44
1200560              32            0.80
1210683             198            4.93
1200548              94            2.34
1200570               5            0.12
1210718             343            8.54
1210734              48            1.19
1200571             332            8.26
1200579              51            1.27
1210770             297            7.39
1200558              30            0.75
1200562             594           14.79
1200554              31            0.77


## **Estrategias para lidar com os valores ausentes**

### Para a Intensidade media do vento optamos por quando a percentagem de valores ausentes for superior a 30% em cada estação, usamos interpolação linear; nas restantes utilizámos a mediana

In [55]:
missing_percent = (df_intesidadeVento10.isnull().mean() * 100)


df_intesidadeVento10_treated = df_intesidadeVento10.copy()

for col in df_intesidadeVento10.columns:
    if missing_percent[col] > 30:
        # Se mais de 30% de ausentes → interpolar
        df_intesidadeVento10_treated[col] = df_intesidadeVento10_treated[col].interpolate(method='linear', limit_direction='both')
        # Preencher o que ainda sobrou (bordas) com a mediana
        mediana = df_intesidadeVento10_treated[col].median()
        df_intesidadeVento10_treated[col] = df_intesidadeVento10_treated[col].fillna(mediana)
    elif missing_percent[col] < 30:
        # Se menos de 5% de ausentes → preencher com mediana
        mediana = df_intesidadeVento10_treated[col].median()
        df_intesidadeVento10_treated[col] = df_intesidadeVento10_treated[col].fillna(mediana)
    else:
        pass

# Resultado final
print("\nResumo dos valores ausentes após tratamento:")
print(df_intesidadeVento10_treated.isnull().sum())


Resumo dos valores ausentes após tratamento:
ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64


In [56]:
os.makedirs('../data/processed', exist_ok=True)

# Salvando o dataset tratado em CSV
df_intesidadeVento10_treated.to_csv('../data/processed/IntensidadeMediaVento10m_processed.csv', index=False)


### Para a temperatura media subtituimos os valores usentes pela media.

In [57]:
df_ttmed = df_temperaturamedia.copy()

In [58]:
media_global = df_ttmed.select_dtypes(include=['float64', 'int64']).stack().mean()

df_ttmed.fillna(media_global, inplace=True)

df_ttmed.to_csv('../data/processed/TemperaturaMedia_processed.csv', index=False)

df_ttmed.isnull().sum()


ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64

### Para a direcao media do vento utilizamos a mediana circular


In [59]:
# Função para calcular a mediana circular (em graus)
def circular_median(values):
    # Converte os valores para radianos
    radians = np.deg2rad(values)

    # Calcula o seno e cosseno da média
    median_radians = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    
    # Converte de volta para graus
    return np.rad2deg(median_radians) % 360

In [60]:
df_ddmed = df_direcaoVento10.copy()

mediana_circular = circular_median(df_ddmed.select_dtypes(include=['float64', 'int64']).stack())

df_ddmed.fillna(mediana_circular, inplace=True)

print("\nResumo dos valores ausentes após o tratamento na folha DD_MED:")
print(df_ddmed.isnull().sum())

df_ddmed.to_csv('../data/processed/DirecaoMediaVento10m_processed.csv', index=False)



Resumo dos valores ausentes após o tratamento na folha DD_MED:
ANO        0
MES        0
DIA        0
1200551    0
1210622    0
1200567    0
1200575    0
1200545    0
1210702    0
1200560    0
1210683    0
1200548    0
1200570    0
1210718    0
1210734    0
1200571    0
1200579    0
1210770    0
1200558    0
1200562    0
1200554    0
dtype: int64


## Agregação dos Dados

### Começar por extrair só o que queremos do Dataset da Repartição da Produção

In [61]:
df_producao_processed = df_producao.copy()

df_producao_processed.columns = df_producao_processed.columns.str.strip()
df_producao_processed = df_producao_processed[['Data e Hora', 'Eólica']]

# Ensure 'Data e Hora' is in datetime format
df_producao_processed['Data e Hora'] = pd.to_datetime(df_producao_processed['Data e Hora'])

df_producao_processed.set_index('Data e Hora', inplace=True)
df_diario = df_producao_processed.resample('D').sum().reset_index()
df_diario.rename(columns={'Data e Hora': 'Data'}, inplace=True)

### Para os datasets do vento temos que mudar o formato da data

In [62]:
df_ddmed_processed = df_ddmed.copy()
df_ddmed_processed = df_ddmed_processed.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})
df_intesidadeVento10_treated = df_intesidadeVento10_treated.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})
df_ddmed_processed['Data'] = pd.to_datetime(df_ddmed_processed[['year', 'month', 'day']])
df_intesidadeVento10_treated['Data'] = pd.to_datetime(df_intesidadeVento10_treated[['year', 'month', 'day']])

# Estações são colunas com nomes numéricos
estacoes = [col for col in df_ddmed_processed.columns if col.isdigit()]

dd_rad = np.deg2rad(df_ddmed_processed[estacoes])
x = df_intesidadeVento10_treated[estacoes] * np.cos(dd_rad)
y = df_intesidadeVento10_treated[estacoes] * np.sin(dd_rad)

x_medio = x.mean(axis=1)
y_medio = y.mean(axis=1)

intensidade_media = np.sqrt(x_medio**2 + y_medio**2)
direcao_media = (np.rad2deg(np.arctan2(y_medio, x_medio))) % 360

df_nacional = pd.DataFrame({
    'Data': df_ddmed_processed['Data'],
    'Intensidade_Media': intensidade_media,
    'Direcao_Media': direcao_media
})

### para a temperatura também

In [63]:
df_ttmed_processed = df_ttmed.copy()

# Ensure the renaming persists by using inplace=True
df_ttmed_processed.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'}, inplace=True)
df_ttmed_processed['Data'] = pd.to_datetime(df_ttmed_processed[['year', 'month', 'day']])

estacoes = [col for col in df_ttmed_processed.columns if col.isdigit()]
temp_media = df_ttmed_processed[estacoes].mean(axis=1)

df_temp = pd.DataFrame({
    'Data': df_ttmed_processed['Data'],
    'Temperatura_Media': temp_media
})

In [64]:
# Juntar os datasets usando a coluna 'Data' como chave
df_merged = df_nacional.merge(df_temp, on='Data', how='inner')
df_merged = df_merged.merge(df_diario, on='Data', how='inner')

df_dados = df_merged.copy()

df_merged.to_csv('../data/processed/agg_data.csv', index=False)

### Dataset para ML

In [ ]:
# --- Carregar os dataframes processados (ou usar os já em memória se preferires) ---
# Assumindo que os passos anteriores (tratamento de ausentes, agregação, merge)
# resultaram num DataFrame chamado 'df_merged' com as colunas:
# 'Data', 'Intensidade_Media', 'Direcao_Media', 'Temperatura_Media', 'Eólica'

# Certificar que está ordenado por data (CRUCIAL para lags e rolling features)
df_merged = df_merged.sort_values('Data').reset_index(drop=True)

# --- Criar Features Adicionais para ML ---

print("A criar features adicionais para ML...")

# 1. Features Baseadas na Data
df_merged['mes'] = df_merged['Data'].dt.month
df_merged['dia_da_semana'] = df_merged['Data'].dt.dayofweek
df_merged['dia_do_ano'] = df_merged['Data'].dt.dayofyear
df_merged['semana_do_ano'] = df_merged['Data'].dt.isocalendar().week.astype(int)
df_merged['trimestre'] = df_merged['Data'].dt.quarter
df_merged['eh_fim_de_semana'] = df_merged['dia_da_semana'].isin([5, 6]).astype(int)

# 2. Codificação Cíclica (Vento e Data)
# Vento (calcular a partir da Direcao_Media diária agregada)
df_merged['vento_sin'] = np.sin(np.radians(df_merged['Direcao_Media']))
df_merged['vento_cos'] = np.cos(np.radians(df_merged['Direcao_Media']))

# Data (usando as features criadas acima)
df_merged['dia_semana_sin'] = np.sin(2 * np.pi * df_merged['dia_da_semana'] / 7)
df_merged['dia_semana_cos'] = np.cos(2 * np.pi * df_merged['dia_da_semana'] / 7)
df_merged['mes_sin'] = np.sin(2 * np.pi * df_merged['mes'] / 12)
df_merged['mes_cos'] = np.cos(2 * np.pi * df_merged['mes'] / 12)
df_merged['dia_ano_sin'] = np.sin(2 * np.pi * df_merged['dia_do_ano'] / 366) # Usar 366 para anos bissextos
df_merged['dia_ano_cos'] = np.cos(2 * np.pi * df_merged['dia_do_ano'] / 366)

# 3. Lag Features (Atrasos)
lags_eolica = [1, 2, 3, 7] # Inclui lag 1 que já tinhas, e adiciona mais
for lag in lags_eolica:
    df_merged[f'Eólica_lag{lag}'] = df_merged['Eólica'].shift(lag)

lags_meteo = [1, 2, 3, 7] # lag 1 já tinhas para Intensidade e Temperatura
for lag in lags_meteo:
    df_merged[f'Intensidade_Media_lag{lag}'] = df_merged['Intensidade_Media'].shift(lag)
    df_merged[f'Temperatura_Media_lag{lag}'] = df_merged['Temperatura_Media'].shift(lag)
    # Lags para componentes do vento
    df_merged[f'vento_sin_lag{lag}'] = df_merged['vento_sin'].shift(lag)
    df_merged[f'vento_cos_lag{lag}'] = df_merged['vento_cos'].shift(lag)

# 4. Rolling Window Features (Janelas Deslizantes)
# Usar shift(1) para garantir que a janela calcula estatísticas sobre dias *anteriores*
window_sizes = [3, 7, 14]
for window in window_sizes:
    # Eólica
    df_merged[f'Eolica_roll_mean_{window}'] = df_merged['Eólica'].shift(1).rolling(window=window, min_periods=1).mean()
    df_merged[f'Eolica_roll_std_{window}'] = df_merged['Eólica'].shift(1).rolling(window=window, min_periods=1).std()
    # Intensidade Média
    df_merged[f'Intensidade_Media_roll_mean_{window}'] = df_merged['Intensidade_Media'].shift(1).rolling(window=window, min_periods=1).mean()
    df_merged[f'Intensidade_Media_roll_std_{window}'] = df_merged['Intensidade_Media'].shift(1).rolling(window=window, min_periods=1).std()
    # Temperatura Média (opcional, pode não ser tão útil)
    # df_merged[f'Temperatura_Media_roll_mean_{window}'] = df_merged['Temperatura_Media'].shift(1).rolling(window=window, min_periods=1).mean()

# 5. Tratar Valores NaN criados pelos Shifts e Rolling Windows
# Preencher NaNs no início usando backfill (usa o próximo valor válido para preencher)
print("Preenchendo valores NaN iniciais com backfill...")
cols_to_fill = [col for col in df_merged.columns if '_lag' in col or '_roll_' in col]
df_merged[cols_to_fill] = df_merged[cols_to_fill].fillna(method='bfill')

# Verificar se ainda existem NaNs (não deve haver se o bfill funcionou)
print("\nVerificação de NaNs após preenchimento:")
print(df_merged.isnull().sum())

# Opcional: Remover colunas originais que foram codificadas ou não são mais necessárias para o modelo
# cols_to_drop = ['Direcao_Media', 'dia_da_semana', 'mes', 'dia_do_ano']
# df_merged = df_merged.drop(columns=cols_to_drop)

# --- Salvar o DataFrame final para ML ---
output_path = '../data/processed/agg_data_ml.csv'
df_merged.to_csv(output_path, index=False)
print(f"\nDataFrame final para ML salvo em: {output_path}")
print("\n--- Primeiras 5 linhas do DataFrame final ---")
print(df_merged.head())
print("\n--- Últimas 5 linhas do DataFrame final ---")
print(df_merged.tail())
print("\n--- Colunas Finais ---")
print(df_merged.columns.tolist())

In [ ]:
df_merged_ann = df_merged.copy()

df_merged['mes'] = df_merged['Data'].dt.month
df_merged['dia_da_semana'] = df_merged['Data'].dt.dayofweek

df_merged['vento_sin'] = np.sin(np.radians(df_merged['Direcao_Media']))
df_merged['vento_cos'] = np.cos(np.radians(df_merged['Direcao_Media']))

df_merged['Intensidade_Media_lag1'] = df_merged['Intensidade_Media'].shift(1)
df_merged['Temperatura_Media_lag1'] = df_merged['Temperatura_Media'].shift(1)
# df_merged['vento_sin_lag1'] = df_merged['vento_sin'].shift(1)
# df_merged['vento_cos_lag1'] = df_merged['vento_cos'].shift(1)

df_merged['Eólica_lag1'] = df_merged['Eólica'].shift(1)
# df_merged['Eólica_lag2'] = df_merged['Eólica'].shift(2)
# df_merged['Eólica_lag3'] = df_merged['Eólica'].shift(3)

In [17]:
df_merged.dropna(inplace=True)

In [18]:
df_merged.isnull().sum()

Data                      0
Intensidade_Media         0
Direcao_Media             0
Temperatura_Media         0
Eólica                    0
mes                       0
dia_da_semana             0
vento_sin                 0
vento_cos                 0
Intensidade_Media_lag1    0
Temperatura_Media_lag1    0
Eólica_lag1               0
dtype: int64

In [ ]:
# Salvar o dataset combinado em um arquivo CSV
# df_merged.to_csv('../data/processed/agg_data_ml.csv', index=False)

### Dataset para ANN

In [20]:
df_merged_ann

,Data,Intensidade_Media,Direcao_Media,Temperatura_Media,Eólica
0,2013-01-01,1.629318,327.262439,9.488889,89358.2
1,2013-01-02,1.564218,31.852420,8.683333,118865.5
2,2013-01-03,2.676138,79.706195,9.277778,232257.7
3,2013-01-04,1.996991,88.710336,8.761111,149543.7
4,2013-01-05,1.045090,83.465882,8.600000,23830.8
...,...,...,...,...,...
4012,2023-12-27,1.607144,113.966469,18.768339,122691.3
4013,2023-12-28,1.329026,132.781290,21.062783,164045.6
4014,2023-12-29,0.992389,105.442474,21.651672,60597.4
4015,2023-12-30,1.090015,160.954414,21.818339,139741.1
